In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from textblob import TextBlob
import sys

# Load data from an Excel file (assuming it contains Twitter data)
df = pd.read_excel(r'C:\Users\RakBank Twitter.xlsx')

### Polarity ### [-1,1]
def sentiscore(x):
    return TextBlob(x).sentiment[0]

### Subjectivity ### [0,1]
def sentimag(x):  
    return TextBlob(x).sentiment[1]

def sentinew(x):  
    return TextBlob(x).sentiment
       
# Rename columns for clarity
df = df.rename(columns={'text': 'message'})
df = df.rename(columns={'created_date': 'created_date'})
df = df.rename(columns={'created_time': 'timestamp'})
df = df.rename(columns={'user.screen_name': 'Company'})

## Change to string
df['message'] = df['message'].astype(str)

## Change to lowercase
messages = df['message'].apply(lambda x: " ".join(x.lower() for x in x.split()))

## Remove Punctuation
messages = messages.str.replace('[^\w\s\@]', '')

## Stemming
st = PorterStemmer()
messages = messages.apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

# Create a DataFrame for sentiment analysis
sentiment_dataframe = pd.DataFrame()

df = df.rename(columns={'retweet_count': 'Retweet Count', 'favorite_count':'Favourite Count'})
df['query_type'].replace('Twitter:/statuses/user_timeline', 'Entires', inplace=True)

## Sentiment Analysis
sentiment_dataframe['messagesscore'] = messages.apply(sentiscore)
sentiment_dataframe['messagesmagnitude'] = messages.apply(sentimag)
sentiment_dataframe['subjectivity/polarity'] = messages.apply(sentinew)

sentiment_dataframe['subjectivity/polarity'] = sentiment_dataframe['subjectivity/polarity'].astype(str)

messages = pd.DataFrame(messages)
parent_id = pd.DataFrame(df['parent_id'])
query_type = pd.DataFrame(df['query_type'])
date = pd.DataFrame(df['created_date'])
time = pd.DataFrame(df['timestamp'])
retweets = pd.DataFrame(df['Retweet Count'])
favourites = pd.DataFrame(df['Favourite Count'])

# Combine all DataFrames
sentiment_dataframe = sentiment_dataframe.join(messages)
sentiment_dataframe = sentiment_dataframe.join(query_type)
sentiment_dataframe = sentiment_dataframe.join(parent_id)
sentiment_dataframe = sentiment_dataframe.join(date)
sentiment_dataframe = sentiment_dataframe.join(time)
sentiment_dataframe = sentiment_dataframe.join(retweets)
sentiment_dataframe = sentiment_dataframe.join(favourites)

# Convert sentiment scores to float
sentiment_dataframe['messagesscore'] = sentiment_dataframe['messagesscore'].apply(lambda x: float(x))

# Create a DataFrame for 'Sentiments'
Sentiment = pd.DataFrame(columns=['Sentiments'])

sentiment_dataframe = sentiment_dataframe.join(Sentiment)

# Classify sentiments based on sentiment scores
sentiment_dataframe['Sentiments'] = np.where((sentiment_dataframe['messagesscore'] > 0).any() and
                                             (sentiment_dataframe['messagesscore'] <= 0.5).any(), 'Good',
                                             sentiment_dataframe['Sentiments'])
sentiment_dataframe['Sentiments'] = np.where(sentiment_dataframe['messagesscore'] == 0, 'Neutral',
                                             sentiment_dataframe['Sentiments'])
sentiment_dataframe['Sentiments'] = np.where(sentiment_dataframe['messagesscore'] > 0.5, 'Very Good',
                                             sentiment_dataframe['Sentiments'])
sentiment_dataframe['Sentiments'] = np.where(sentiment_dataframe['messagesscore'] < 0, 'Bad',
                                             sentiment_dataframe['Sentiments'])
sentiment_dataframe['Sentiments'] = np.where(sentiment_dataframe['messagesscore'] < -0.5, 'Very Bad',
                                             sentiment_dataframe['Sentiments'])

sentiment_dataframe = sentiment_dataframe.iloc[1:, :]

# Save the resulting DataFrame to a CSV file
sentiment_dataframe.to_csv(r'C:\Users\RakBank_TwitterSentiments.csv', index=None, header=True)
